In [1]:
def process_bnc_tagged(filename="/content/bnc_tagged.txt"):
  """'bnc_tagged.txt' 파일을 읽고 처리한 후, 결과를 반환합니다.

  입력 데이터 형식:
  - 읽고 처리할 코퍼스는 의존구조 파싱(dependency parsing)이 된 파일
  - 각 문장은 '<s id=숫자>'로 시작하고 '</s>'로 끝남.
  - 문장의 각 줄은 하나의 토큰을 나타나며, 탭으로 구분된 여러 정보를 포함함 (7열 탭 구분 데이터): 단어 형태, 품사(UPOS), 기본형(lemma), 단어 ID, 핵어 ID(head ID), 핵어(head word), 의존관계(deprel)

  함수의 작동:
  - 각 문장에서 탭으로 구분된 7개의 값으로 이루어진 단어 정보들을 모아서 리스트로 만든다.
  - 문장 ID 와 단어 정보 리스트를 튜플로 묶어서 결과 리스트에 추가한다. 단어 정보가 7개의 값을 갖지 않으면 그 단어는 건너뛴다.

  반환 (출력 데이터): (문장 ID, 토큰 리스트) 형태의 튜플을 저장한 리스트
  """
  results = []
  with open(filename, 'r', encoding='utf-8') as f:
    current_sentence_id = None
    current_sentence_tokens = []
    for line in f:
      line = line.strip()
      if line.startswith('<s id='):
        # 새 문장 시작
        if current_sentence_id is not None:
          results.append((current_sentence_id, current_sentence_tokens))
        current_sentence_id = line[5:-1]  # '<s id=숫자>'에서 숫자만 추출
        current_sentence_tokens = []
      elif line == '</s>':
        # 문장 끝
        results.append((current_sentence_id, current_sentence_tokens))
        current_sentence_id = None
        current_sentence_tokens = []
      else:
        # 단어 정보 처리
        token_info = line.split('\t')
        if len(token_info) == 7:
          current_sentence_tokens.append(token_info)
  return results

In [2]:
def create_verb_dictionary(filename="/content/verb_list.txt"):
  """'verb_list.txt' 파일을 읽고, 동사 딕셔너리를 생성하여 반환합니다.

  입력 데이터 형식:
  - 파일의 각 줄에는 코퍼스로부터 용례를 추출하려는 동사의 기본형(lemma)이 적혀 있음.

  함수의 작동:
  - 파일의 각 줄에서 읽어온 문자열(동사 기본형)을 key로 하고, 빈 리스트 '[]'를 value로 해서 딕셔너리에 추가한다.

  반환 (출력 데이터):
  - 동사를 key로 하고 빈 리스트를 value로 하는 딕셔너리
  """
  verb_list = {}
  with open(filename, 'r', encoding='utf-8') as f:
    for line in f:
      verb = line.strip()  # 각 줄에서 동사 기본형 추출
      verb_list[verb] = []  # 동사를 key로, 빈 리스트를 value로 딕셔너리에 추가
  return verb_list

In [3]:
from collections import defaultdict

def extract_verb_examples(corpus_data, verb_dictionary):
  """코퍼스 데이터에서 본동사 용례를 추출하여 동사 딕셔너리에 추가합니다.

  Args:
    corpus_data: (문장 ID, 토큰 리스트) 형태의 튜플을 담은 리스트
    verb_dictionary: 동사를 key로 하고, 빈 리스트를 value로 갖는 defaultdict(list) 형태

  Returns:
    동사 용례 딕셔너리
  """

  for sentence_id, tokens in corpus_data:
    for token in tokens:
      lemma = token[2]  # 기본형
      pos = token[1]  # 품사
      deprel = token[6]  # 의존관계

      if lemma in verb_dictionary and pos == "VERB" and deprel == "root":
        sentence_text = " ".join([t[0] for t in tokens])  # 문장 단어 연결
        verb_dictionary[lemma].append((sentence_id, sentence_text))  # 용례 추가

  return verb_dictionary

In [4]:
from collections import defaultdict

def extract_passive_verb_examples(corpus_data, verb_dictionary):
  """코퍼스 데이터에서 수동태 구문에 사용된 동사 용례를 추출하여 동사 딕셔너리에 추가합니다.

  Args:
    corpus_data: (문장 ID, 토큰 리스트) 형태의 튜플을 담은 리스트
    verb_dictionary: 동사를 key로 하고, 빈 리스트를 value로 갖는 defaultdict(list) 형태

  Returns:
    동사 용례 딕셔너리
  """
  passive_verb_examples = defaultdict(list)

  for sentence_id, tokens in corpus_data:
    for i, token in enumerate(tokens):
      lemma = token[2]  # 기본형
      pos = token[1]  # 품사
      deprel = token[6]  # 의존관계
      head_id = token[4]  # head ID

      # 조건 1: 기본형이 동사 딕셔너리 key에 포함, 품사가 VERB, 의존관계가 root
      if lemma in verb_dictionary and pos == "VERB" and deprel == "root":
        # 조건 4: aux:pass 의존관계를 갖는 의존소 확인
        has_aux_pass = any(t[6] == "aux:pass" and t[4] == str(i + 1) for t in tokens)

        # 조건 5: compound:prt 의존관계를 갖는 의존소 확인 (없어야 함)
        has_compound_prt = any(t[6] == "compound:prt" and t[4] == str(i + 1) for t in tokens)

        # 모든 조건을 만족하는 경우
        if has_aux_pass and not has_compound_prt:
          sentence_text = " ".join([t[0] for t in tokens])  # 문장 단어 연결
          passive_verb_examples[lemma].append((sentence_id, sentence_text))  # 용례 추가

  return passive_verb_examples

In [5]:
from collections import defaultdict

def extract_transitive_verb_examples(corpus_data, verb_dictionary):
  """코퍼스 데이터에서 타동사 용례를 추출하여 동사 딕셔너리에 추가합니다.

  Args:
    corpus_data: (문장 ID, 토큰 리스트) 형태의 튜플을 담은 리스트
    verb_dictionary: 동사를 key로 하고, 빈 리스트를 value로 갖는 defaultdict(list) 형태

  Returns:
    동사 용례 딕셔너리
  """
  transitive_verb_examples = defaultdict(list)

  for sentence_id, tokens in corpus_data:
    for i, token in enumerate(tokens):
      lemma = token[2]  # 기본형
      pos = token[1]  # 품사
      deprel = token[6]  # 의존관계
      #head_id = token[4]  # head ID

      # 조건 1: 기본형이 동사 딕셔너리 key에 포함, 품사가 VERB, 의존관계가 root
      if lemma in verb_dictionary and pos == "VERB" and deprel == "root":
        # 조건 4: obj 의존관계를 갖는 의존소 확인
        has_obj = any(t[6] == "obj" and t[4] == str(i + 1) for t in tokens)

        # 조건 5: aux:pass 또는 compound:prt 의존관계를 갖는 의존소 확인 (없어야 함)
        has_aux_pass = any(t[6] == "aux:pass" and t[4] == str(i + 1) for t in tokens)
        has_compound_prt = any(t[6] == "compound:prt" and t[4] == str(i + 1) for t in tokens)

        # 모든 조건을 만족하는 경우
        if has_obj and not has_aux_pass and not has_compound_prt:
          sentence_text = " ".join([t[0] for t in tokens])  # 문장 단어 연결
          transitive_verb_examples[lemma].append((sentence_id, sentence_text))  # 용례 추가

  return transitive_verb_examples

In [11]:
from collections import defaultdict

def extract_intransitive_verb_examples(corpus_data, verb_dictionary):
  """코퍼스 데이터에서 자동사 용례를 추출하여 동사 딕셔너리에 추가합니다.

  Args:
    corpus_data: (문장 ID, 토큰 리스트) 형태의 튜플을 담은 리스트
    verb_dictionary: 동사를 key로 하고, 빈 리스트를 value로 갖는 defaultdict(list) 형태

  Returns:
    동사 용례 딕셔너리
  """
  intransitive_verb_examples = defaultdict(list)

  for sentence_id, tokens in corpus_data:
    for i, token in enumerate(tokens):
      lemma = token[2]  # 기본형
      pos = token[1]  # 품사
      deprel = token[6]  # 의존관계

      # 조건 1: 기본형이 동사 딕셔너리 key에 포함, 품사가 VERB, 의존관계가 root
      if lemma in verb_dictionary and pos == "VERB" and deprel == "root":
        # 조건 4: obj, aux:pass, compound:prt 의존관계를 갖는 의존소 확인 (없어야 함)
        has_obj = any(t[6] == "obj" and t[4] == str(i + 1) for t in tokens)
        has_aux_pass = any(t[6] == "aux:pass" and t[4] == str(i + 1) for t in tokens)
        has_compound_prt = any(t[6] == "compound:prt" and t[4] == str(i + 1) for t in tokens)

        # 모든 조건을 만족하는 경우 (obj, aux:pass, compound:prt가 없는 경우)
        if not has_obj and not has_aux_pass and not has_compound_prt:
          sentence_text = " ".join([t[0] for t in tokens])  # 문장 단어 연결
          intransitive_verb_examples[lemma].append((sentence_id, sentence_text))  # 용례 추가

  return intransitive_verb_examples

In [12]:
def save_verb_examples(verb_examples_dict, folder, header, type):
  """동사 용례 딕셔너리를 두 개의 텍스트 파일로 저장합니다.

  Args:
    verb_examples_dict: 동사를 key로, 용례 리스트((문장 ID, 문장 문자열) 튜플)를 value로 갖는 딕셔너리
    folder: 출력 파일을 저장할 폴더 이름
    header: 출력 파일 이름 앞부분에 사용할 문자열
    type: 용례 유형을 나타내는 문자열 (all, passive, transitive 등)
  """
  import os

  # 폴더 생성
  os.makedirs(folder, exist_ok=True)

  # 파일 이름 생성
  count_filename = os.path.join(folder, f"{header}_word_{type}_count.txt")
  examples_filename = os.path.join(folder, f"{header}_word_{type}.txt")

  # 빈도수 파일 저장
  with open(count_filename, 'w', encoding='utf-8') as count_file:
    for verb, examples in verb_examples_dict.items():
      # 중복 제거
      unique_examples = list(set(examples))
      count = len(unique_examples)
      if count > 0:
        first_example = unique_examples[0][1]  # 첫 번째 용례 문장
        count_file.write(f"{verb}\t{count}\t{first_example}\n")

  # 용례 파일 저장
  with open(examples_filename, 'w', encoding='utf-8') as examples_file:
    for verb, examples in verb_examples_dict.items():
      for sentence_id, sentence_text in examples:
        examples_file.write(f"{verb}\t{sentence_id}\t{sentence_text}\n")

In [13]:
import json
from collections import defaultdict
import os # import os module for file and folder operations


# 이전 단계에서 정의한 함수들 가져오기
# process_bnc_tagged, create_verb_dictionary, extract_verb_examples,
# extract_passive_verb_examples, extract_transitive_verb_examples,
# extract_intransitive_verb_examples, save_verb_examples

# explicitly import required functions from other modules/cells
from IPython.utils import io # Added to handle potential io errors

# The with statement was incorrectly placed and is not needed in this case
# with io.capture_output() as captured: #Added to handle potential io errors


#import extract_intransitive_verb_examples from the relevant cell/module if they are defined there.
#from . import extract_intransitive_verb_examples  # example assuming the function is in the same package
#or
#import notebook_name.module_name as module_containing_function
#extract_intransitive_verb_examples = module_containing_function.extract_intransitive_verb_examples

# if they are defined in separate notebook cells (assuming you are using notebooks)
# you can call them directly, you don't need a separate import
# but you need to make sure those cells have been run before this cell is executed.


# assuming the other functions are also in separate cells and those cells are run:
# from . import process_bnc_tagged
# from . import create_verb_dictionary
# from . import extract_verb_examples
# from . import extract_passive_verb_examples
# from . import extract_transitive_verb_examples
# from . import save_verb_examples

def main():
    """
    'bnc_tagged.txt'와 'verb_list.txt'를 입력으로 받아 동사 용례를 추출하고 저장하는 메인 함수입니다.
    """
    corpus_file = "/content/bnc_tagged.txt"
    verb_list_file = "/content/verb_list.txt"
    output_folder = "output"

    # 1. 코퍼스 전처리
    corpus_data = process_bnc_tagged(filename=corpus_file)

    # 2. 동사 목록 전처리
    verb_dictionary = create_verb_dictionary(filename=verb_list_file)

    # 3. 용례 추출
    all_verb_examples = extract_verb_examples(corpus_data, verb_dictionary.copy())
    passive_verb_examples = extract_passive_verb_examples(corpus_data, verb_dictionary.copy())
    transitive_verb_examples = extract_transitive_verb_examples(corpus_data, verb_dictionary.copy())
    intransitive_verb_examples = extract_intransitive_verb_examples(corpus_data, verb_dictionary.copy())

    # 4. 결과 처리
    save_verb_examples(all_verb_examples, output_folder, "bnc", "all")
    save_verb_examples(passive_verb_examples, output_folder, "bnc", "passive")
    save_verb_examples(transitive_verb_examples, output_folder, "bnc", "transitive")
    save_verb_examples(intransitive_verb_examples, output_folder, "bnc", "intransitive")

    # 5. 코퍼스 전처리 결과 저장
    with open(os.path.join(output_folder, "bnc_corpus_data.txt"), 'w', encoding='utf-8') as f:
        for sentence_id, tokens in corpus_data:
            f.write(f"{sentence_id}\t{' '.join([t[0] for t in tokens])}\n")  # 문장 ID와 문장 텍스트를 탭으로 구분하여 저장

    # 6. 동사 목록 전처리 결과 저장
    with open(os.path.join(output_folder, "verb_dictionary.txt"), 'w', encoding='utf-8') as f:
        json.dump(verb_dictionary, f, indent=2)  # JSON 형식으로 저장

if __name__ == "__main__":
    main()